In [5]:
import re
import requests
from bs4 import BeautifulSoup

In [7]:
full_review = list()
counts = 0 
for i in range(1, 10):
    ready_response = requests.get(f'https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/?sortby=post_date%3ADesc&pagesize=100')
    if ready_response.status_code == 200:
        ready_soup = BeautifulSoup(ready_response.text, 'html.parser')  
    else:
        print(f"Error: Unable to fetch the content:{ready_response.status_code}")

    all_article_in_a_page = ready_soup.find_all('article', {"itemprop": "review"})
    pattern = r'\((.*?)\)'

    for _, article in enumerate(all_article_in_a_page):
        table = article.find('div', {'class': 'body'}).find('div').find('div', {'class': 'review-stats'}).find('table').find_all('tr')
        
        review_dict = {}
        for tr in table:
            header_td = tr.find('td', class_='review-rating-header')
            value_td = tr.find('td', class_='review-value')
            stars_td = tr.find('td', {'class': 'review-rating-stars'})
            
            if header_td and value_td:
                header = header_td.get_text(strip=True)
                value = value_td.get_text(strip=True)
                review_dict[header] = value
            elif not value_td:
                header = header_td.get_text(strip=True)
                stars = stars_td.get_text(strip=True)
                fill_stars = stars_td.find_all('span', class_='fill')
                review_dict[header] = len(fill_stars)

        
        # is_verified = article.find('div', {'class': 'body'}).find('div', {'class': 'tc_mobile'}).find('div', {'class': 'text_content'}).find('strong')   
        # is_verified_ready = is_verified.text if is_verified is not None else None
        # country_of_origin = re.findall(pattern, article.find('div', {'class': 'body'}).find('h3').text)[0]
        # country_of_origin_ready = country_of_origin.text if country_of_origin is not None else None
        # detailed_review = article.find('div', {'class': 'body'}).find('div').find('div', {'class': 'text_content'}).text.split('|')[1].lstrip() 
        # detailed_review_ready = detailed_review.text if detailed_review is not None else None
        
        
        # full_review.append({
        #     'id': counts,
        #     'review_header': article.find('div', {'class': 'body'}).find('h2', {'class': 'text_header'}).text.strip('"'),
        #     'date': article.find('div', {'class': 'body'}).find('h3').find('time').text,
        #     'country_of_origin': country_of_origin,
        #     'passenger_name': article.find('div', {'class': 'body'}).find('h3').find('span').text.removeprefix('\n').removesuffix('\n').split('\n')[-1],
        #     'is_verified': is_verified,
        #     'detailed_review': detailed_review,
        #     'rating_dict': review_dict,
        # })

        try:
            full_review.append({
                'id': counts,
                'review_header': article.find('div', {'class': 'body'}).find('h2', {'class': 'text_header'}).text.strip('"'),
                'date': article.find('div', {'class': 'body'}).find('h3').find('time').text,
                'country_of_origin': re.findall(pattern, article.find('div', {'class': 'body'}).find('h3').text)[0],
                'passenger_name': article.find('div', {'class': 'body'}).find('h3').find('span').text.removeprefix('\n').removesuffix('\n').split('\n')[-1],
                'is_verified': article.find('div', {'class': 'body'}).find('div', {'class': 'tc_mobile'}).find('div', {'class': 'text_content'}).find('strong').text,
                'detailed_review': article.find('div', {'class': 'body'}).find('div').find('div', {'class': 'text_content'}).text.split('|')[1].lstrip(),
                'rating_dict': review_dict,
            })
        except:
            full_review.append(None)
    
        counts+=1
    

In [119]:
article_dict = list()
counts = 0 
for i in range(1,30):
    response = requests.get(f'https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/?sortby=post_date%3ADesc&pagesize=100')
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')  
    else:
        print(f"Error: Unable to fetch the content:{response.status_code}")

    all_article_in_a_page = soup.find_all('article', {"itemprop": "review"})
    pattern = r'\((.*?)\)'
    get_int_pattern = r'\d+'
    
    for i, article in enumerate(all_article_in_a_page):
        table = article.find('div', {'class': 'body'}).find('div').find('div', {'class': 'review-stats'}).find('table').find_all('tr')
        
        review_dict = {}
        for tr in table:
            header_td = tr.find('td', class_='review-rating-header')
            value_td = tr.find('td', class_='review-value')
            stars_td = tr.find('td', {'class': 'review-rating-stars'})
            
            if header_td and value_td:
                header = header_td.get_text(strip=True)
                value = value_td.get_text(strip=True)
                review_dict[header] = value
            elif not value_td:
                header = header_td.get_text(strip=True)
                stars = stars_td.get_text(strip=True)
                fill_stars = stars_td.find_all('span', class_='fill')
                review_dict[header] = len(fill_stars)

        author = article.find('div', {'class': 'body'}).find('h3').find('span', {'itemprop': 'author'})
        if author.a is not None:
            passenger_name = author.find('span', {'itemprop': 'name'}).text
            no_of_reviews = int(re.findall(get_int_pattern, author.a.span.text)[0])
        elif author.a is None:
            passenger_name = author.text.removeprefix('\n').removesuffix('\n').split('\n')[-1]
            no_of_reviews = 0
            
        # .find('strong').text
        text_content = article.find('div', {'class': 'body'}).find('div').find('div', {'class': 'text_content'})
        if text_content.strong is not None:
            is_verified = text_content.find('strong').text
            detailed_review = text_content.text
        elif text_content.strong is None:
            is_verified = 'not specified'
            detailed_review = text_content.text
            
        article_dict.append({
            'id': counts,
            'review_header': article.find('div', {'class': 'body'}).find('h2', {'class': 'text_header'}).text.strip('"'),
            'no_of_reviews': no_of_reviews,
            'passenger_name': passenger_name,
            'country_of_origin': re.findall(pattern, article.find('div', {'class': 'body'}).find('h3').text)[0],
            'date': article.find('div', {'class': 'body'}).find('h3').find('time').text,
            'is_verified': is_verified,
            'detailed_review': detailed_review,
            'rating_dict': review_dict,
        })
    
        counts+=1

In [120]:
# text = '12, int'
# integer = re.findall(r'\d+', text)
# int(integer[0])

In [121]:
len(article_dict)

2900

In [113]:
from pprint import pprint
pprint(article_dict)

[{'country_of_origin': 'Portugal',
  'date': '13th March 2018',
  'detailed_review': 'Two regular an uneventful flights. Curiously enough, '
                     'though, with the exact same crew! The crew were very '
                     'nice and the service is very attentive and polite, but I '
                     'just cannot take it that British Airways has chosen to '
                     'provide a service just like low cost carriers, where '
                     'everything is charged for, apart from luggage. On the '
                     'second leg of the trip, the aircraft felt extremely warm '
                     'and for some odd reason, row 6 where I was sitting did '
                     'not have AC.',
  'id': 0,
  'is_verified': 'not specified',
  'no_of_reviews': 0,
  'passenger_name': 'Angelo Menezes',
  'rating_dict': {'Cabin Staff Service': 4,
                  'Date Flown': 'March 2018',
                  'Food & Beverages': 1,
                  'Ground Service'